This is an updated version of my wikipedia scraper, the notebook explaining the first version can be found [here](https://github.com/grumpyclimber/portfolio/blob/main/wiki_scrape/scrape_wiki.ipynb). In essence the function takes an 'url friendly' (no spaces, just underscores) movie title and generates a search query on Wikipedia. Then scrapes the results of search results to get the url of the movie's page. Once we have the url of the actual page we can scrape some data from it.

The updated version changes:
* checking if the result description contains the word 'film' (step. 3.1.) and scraping the adequate result
* scraping the distribution company name (step 7)
* option of adding additional keyword to search query (default path_tail='_movie')

In [1]:
def scrape_wiki_vol2(df, path_tail='_movie', number_to_add=0):
    # 1. Input:
    search_query = df['title_urled'] + path_tail
    # 2. Put the title into wikipedia search and extract the link to the first result:(it's not the first link!!!)
    url = "https://en.wikipedia.org/w/index.php?search="+search_query+"&title=Special:Search&profile=advanced&fulltext=1&ns0=1"
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    web_links = soup.find_all("a")
    # 3 check if search results page is empty or not - if the page has less than 43 links - it's an empty page
    # with no results!
    if len(web_links)>43:
        spans = soup.find_all("div", class_="searchresult")
        n = 0
        # 3.1. check descriptions of results, we're after a result with a word 'film' in the description
        for el in range(len(spans)):
            if spans[el].text.find('film') != -1:
                n=el
                break
        # 4. the first result of our seach query is actually the tenth link on the results page...normally
        # but sometimes it's further down the list (that's where we add the number from step 3.1)
        movie_path = web_links[10+n+number_to_add].get("href")   
        # 5. now lets scrape all of the infobox-labels into a list a check how long is that list:
        response = requests.get("https://en.wikipedia.org"+movie_path)
        content = response.content
        parser = BeautifulSoup(content, 'html.parser')
        par_len = len(parser.find_all("th", class_="infobox-label"))
        list_of_tags = parser.find_all("th", class_="infobox-label")
        # 6. Loop trough infobox-labels list and find the position of 'Budget':
        for num in range(8,par_len):        
            tag_name = list_of_tags[num]
            if tag_name.text == 'Budget':  
                tag_numbers = parser.find_all("td", class_="infobox-data")[num]
                cash = tag_numbers.text
                break
        # 6.1. If we can't find the budget:
        else:
            cash = None
        # 7. same for distributor:                
        for num in range(4,par_len):        
            tag_name = list_of_tags[num]
            if tag_name.text == 'Distributed by':  
                tag_numbers = parser.find_all("td", class_="infobox-data")[num]
                distributor = tag_numbers.text
                break
        # 7.1. If we can't find the distributor:
        else:
            distributor = None
        return cash, distributor